In [1]:
import pandas as pd
import numpy as np
import os

import logging
logging.getLogger().setLevel(logging.INFO)

from alphabase.spectral_library.flat import SpecLibFlat
from alphabase.spectral_library.base import SpecLibBase
from alphabase.spectral_library.reader import SWATHLibraryReader
from alphadia.extraction.utils import reannotate_fragments
from alphabase.psm_reader import psm_reader_provider

INFO:root:
0:00:00.873124  INFO: Cupy is not available


In [2]:
lib_location = '/Users/georgwallmann/Documents/data/raw_data/Alpha_dia_benchmarking/dia-PASEF_library/21min_Evosep_HeLa_BR14_48fractions_diaPASEF_py_diAID_2.tsv'
#insilico_lib_location = '/Users/georgwallmann/Documents/data/libraries/2022_111_brunner_2022_1ng_all_29_human_trypsin_diann_decoy.hdf'
output_location = '/Users/georgwallmann/Documents/data/raw_data/Alpha_dia_benchmarking/diaPASEF_vs_synchroPASEF/library_48_fractions_MSFragger.hdf'

In [3]:
# create dense library from diann psm file
target_lib = SWATHLibraryReader()
target_lib.import_file(lib_location)
target_lib.precursor_df['decoy'] = 0
#target_lib._fragment_mz_df = target_lib._fragment_mz_df[['b_z1', 'b_z2', 'y_z1', 'y_z2']]
#target_lib._fragment_intensity_df = target_lib._fragment_intensity_df[['b_z1', 'b_z2', 'y_z1', 'y_z2']]

target_lib._precursor_df['elution_group_idx'] = np.arange(len(target_lib._precursor_df))

/Users/georgwallmann/Documents/git/alphabase/alphabase/spectral_library/reader.py:193: DtypeWarning: Columns (33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, sep=self.csv_sep)


Index(['ReferenceRun', 'PrecursorCharge', 'Workflow', 'IntModifiedPeptide',
       'CV', 'AllowForNormalization', 'ModifiedPeptide', 'StrippedPeptide',
       'iRT', 'IonMobility', 'iRTSourceSpecific', 'BGSInferenceId',
       'IsProteotypic', 'IntLabeledPeptide', 'LabeledPeptide', 'PrecursorMz',
       'ReferenceRunQvalue', 'ReferenceRunMS1Response', 'FragmentLossType',
       'FragmentNumber', 'FragmentType', 'FragmentCharge', 'FragmentMz',
       'RelativeIntensity', 'ExcludeFromAssay', 'Database', 'ProteinGroups',
       'UniProtIds', 'Protein Name', 'ProteinDescription', 'Organisms',
       'OrganismId', 'Genes', 'Protein Existence', 'Sequence Version',
       'FASTAName'],
      dtype='object')


In [11]:
target_lib.precursor_df

,raw_name,sequence,charge,rt,mobility,proteins,mods,mod_sites,nAA,frag_start_idx,frag_stop_idx,rt_norm,precursor_mz,ccs,decoy,elution_group_idx,mod_seq_hash,mod_seq_charge_hash
0,20220715_TIMS03_PaSk_SA_HeLa_Evo05_21min_IM07-...,AAAAAAAAAAAAAAAGAGAGAK,2,44.969955,1.218,P55011,,,22,0,21,0.698307,798.926260,492.064062,0,0,4054020141821570781,4054020141821570783
1,20220715_TIMS03_PaSk_SA_HeLa_Evo05_21min_IM07-...,AAAAAAAAAAAAASVGGTIPGPGPGGGQGPGEGGEGQTAR,3,63.065834,1.154,Q76L83,,,40,21,60,0.775191,1086.534641,696.236084,0,1,2969858240496417430,2969858240496417433
2,20220715_TIMS03_PaSk_SA_HeLa_Evo05_21min_IM07-...,AAAAAAAAAAAAASVGGTIPGPGPGGGQGPGEGGEGQTAR,4,63.065834,1.141,Q76L83,,,40,60,99,0.775191,815.152800,917.855953,0,2,2969858240496417430,2969858240496417434
3,20220715_TIMS03_PaSk_SA_HeLa_Evo05_21min_IM07-...,AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGP...,4,53.954920,1.221,O60341,,,52,99,150,0.711829,1101.283999,981.123454,0,3,-3933421607492937340,-3933421607492937336
4,20220715_TIMS03_PaSk_SA_HeLa_Evo05_21min_IM07-...,AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGP...,5,53.954920,1.106,O60341,,,52,150,201,0.711829,881.228655,1110.894509,0,4,-3933421607492937340,-3933421607492937335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235317,20220715_TIMS03_PaSk_SA_HeLa_Evo05_21min_IM07-...,VSNPVHGLPFLPGTSFK,2,97.816650,1.158,Q5JVL4,Acetyl@Protein N-term,0,17,3085029,3085045,0.857850,919.993611,467.294035,0,235317,-2937157882908476566,-2937157882908476564
235318,20220715_TIMS03_PaSk_SA_HeLa_Evo05_21min_IM07-...,VSVINTVDTSHEDMIHDAQMDYYGTR,3,79.621310,1.094,P55735,Acetyl@Protein N-term,0,26,3085045,3085070,0.828414,1013.786810,660.238296,0,235318,-7992903986759402260,-7992903986759402257
235319,20220715_TIMS03_PaSk_SA_HeLa_Evo05_21min_IM07-...,VSVINTVDTSHEDMIHDAQMDYYGTR,3,73.789665,1.091,P55735,Acetyl@Protein N-term;Oxidation@M,0;20,26,3085070,3085095,0.795534,1019.118448,658.412059,0,235319,-2746682706757545470,-2746682706757545467
235320,20220715_TIMS03_PaSk_SA_HeLa_Evo05_21min_IM07-...,VSVINTVDTSHEDMIHDAQMDYYGTR,3,66.958626,1.104,P55735,Acetyl@Protein N-term;Oxidation@M,0;14,26,3085095,3085120,0.806313,1019.118448,666.257482,0,235320,4383791154456492051,4383791154456492054


In [12]:
decoy_lib = SpecLibBase()
decoy_lib._precursor_df = target_lib.precursor_df.copy()
decoy_lib._fragment_mz_df = target_lib._fragment_mz_df.copy()
decoy_lib._fragment_intensity_df = target_lib._fragment_intensity_df.copy()

#decoy_lib.import_file(brunner_lib)
decoy_lib.decoy = 'diann'
decoy_lib.append_decoy_sequence()
decoy_lib.calc_precursor_mz()
decoy_lib.precursor_df = decoy_lib.precursor_df[decoy_lib.precursor_df['decoy'] == 1]

In [13]:
from peptdeep.pretrained_models import ModelManager
from alphabase.peptide.fragment import get_charged_frag_types

frag_types = get_charged_frag_types(
    ['b','y'], 
    2
)

model_mgr = ModelManager()

model_mgr.nce = 30
model_mgr.instrument = 'timsTOF'

In [14]:
res = model_mgr.predict_all(
    decoy_lib.precursor_df,
    predict_items=['ms2'],
    frag_types = frag_types,
)

2023-02-15 13:52:09> Using multiprocessing with 8 processes ...
2023-02-15 13:52:09> Predicting ms2 ...


100%|██████████| 46/46 [00:49<00:00,  1.08s/it]


In [15]:
decoy_lib._precursor_df = res['precursor_df']
decoy_lib._fragment_mz_df = res['fragment_mz_df']
decoy_lib._fragment_intensity_df = res['fragment_intensity_df']

In [16]:
decoy_lib._precursor_df[['frag_start_idx']] += len(target_lib._fragment_mz_df)
decoy_lib._precursor_df[['frag_stop_idx']] += len(target_lib._fragment_mz_df)

In [17]:
output_lib = SpecLibBase()

output_lib._precursor_df = pd.concat([target_lib._precursor_df, decoy_lib._precursor_df], join='inner')
output_lib._fragment_mz_df = pd.concat([target_lib._fragment_mz_df, decoy_lib._fragment_mz_df], join='inner')
output_lib._fragment_intensity_df = pd.concat([target_lib._fragment_intensity_df, decoy_lib._fragment_intensity_df], join='inner')

In [18]:
output_lib._precursor_df = output_lib._precursor_df.sort_values(by=['elution_group_idx']).reset_index(drop=True)
output_lib.precursor_df['precursor_idx'] = np.arange(len(output_lib.precursor_df))

In [19]:
from tqdm import tqdm
from alphabase.peptide.precursor import get_mod_seq_formula
from alphabase.constants.isotope import IsotopeDistribution


def calc_isotope_dist(precursor_df, max_isotope = 6, min_intensity = 0.001):
    isotope_dist = IsotopeDistribution()

    col_names = ['i_{}'.format(i) for i in range(max_isotope)]
    precursor_df[col_names] = 0.

    for i in tqdm(range(len(precursor_df))):

        row = precursor_df.iloc[i]
        dist, mono = isotope_dist.calc_formula_distribution(
            get_mod_seq_formula(row['sequence'], row['mods'])
        )
        dist[dist < 0.001] = 0.
        dist = dist / dist.sum()
        precursor_df.loc[i, col_names] = dist[:max_isotope]

In [20]:
calc_isotope_dist(output_lib.precursor_df)

100%|██████████| 465303/465303 [02:52<00:00, 2696.36it/s]


In [21]:
output_lib._precursor_df.drop(columns=['rt'], inplace=True)

In [22]:
output_lib.save_hdf(output_location)

In [27]:
output_lib._precursor_df.iloc[2:5][['sequence', 'charge', 'decoy','mod_seq_charge_hash']]

,sequence,charge,decoy,mod_seq_charge_hash
2,AAAAAAAAAAAAASVGGTIPGPGPGGGQGPGEGGEGQTAR,3,0,2969858240496417433
3,ALAAAAAAAAAAASVGGTIPGPGPGGGQGPGEGGEGQTLR,3,1,2969858240496417433
4,AAAAAAAAAAAAASVGGTIPGPGPGGGQGPGEGGEGQTAR,4,0,2969858240496417434
